##add_phasing_to_read

Takes in an aligned bam file and writes Jason's phasing information to the read based on the network graph defintion

In [1]:
#init
%run ipy_setup.py
%matplotlib inline

In [4]:
import pysam
aln_bamfile = pysam.AlignmentFile('/hpc/users/neffr01/jason_new/falcon_unzip_jason_071715/3-falcon_unzip/000000F/aln_sorted.bam', 'rb')
out_fp = pysam.AlignmentFile('/hpc/users/neffr01/jason_new/falcon_unzip_jason_071715/3-falcon_unzip/000000F/aln_sorted.ann.bam', 'wb', template=aln_bamfile)
hap_info = pd.read_csv('/hpc/users/neffr01/jason_new/falcon_unzip_jason_071715/3-falcon_unzip/000000F/phased_reads', sep=' ', header=None)

# index hap info file
hap_info.set_index(hap_info[5], inplace=True)
phased_set = frozenset(hap_info[5])

In [25]:
count = 0
for bamread in aln_bamfile.fetch():
    count += 1
    if (count % 100) == 0:
        sys.stdout.write("\rRead %s lines from input." % str(count))
        sys.stdout.flush()
    if bamread.query_name in phased_set:
        readline = hap_info.ix[bamread.query_name]
        hapblock = str(readline[1])
        haplotype = str(readline[2])
        bamread.tags += [("ZF", hapblock + "," + haplotype)] # add info
        out_fp.write(bamread)
    else:
        out_fp.write(bamread)

out_fp.close()
aln_bamfile.close()

Read 290800 lines from input.